### This file loads, merges and cleans Eden Tenant Data 

In [1]:
import pandas as pd
import numpy as np
import os
import xlrd
import re
from functools import reduce
from itertools import groupby
from matplotlib import pyplot as plt
import pandas_dedupe
plt.show()
import seaborn as sns; sns.set()
import altair as alt
alt.data_transformers.disable_max_rows()
import warnings
warnings.filterwarnings('ignore')
from fuzzywuzzy import fuzz
pd.options.display.float_format = '{:}'.format #get rid of scientific notation 

import eden_data_functions #this .py file holds the majority of the functions for this analysis 
pd.set_option('display.max_columns', None)
%load_ext autoreload
%autoreload 2

Get and clean sheet names from excel files

In [5]:
may2020_sept2020_location = "/Users/quinnunderriner/Desktop/Work/eden_rent_roll/TenantLeasePayments_Trending_(Active)_May_2020_Sept_2020.xlsx"
may2019_sept2019_location = "/Users/quinnunderriner/Desktop/Work/eden_rent_roll/Tenant_Lease_Payments_May 2019_Sept_2019.xlsx"

other_months_2020_location = "/Users/quinnunderriner/Desktop/Work/eden_rent_roll/Eden_2020_ Tenant_Lease_Payments_.zactive.xlsx"
other_months_2019_location = "/Users/quinnunderriner/Desktop/Work/eden_rent_roll/Eden_2019_ Tenant_Lease_Payments_.zactive.xlsx"

In [12]:
may2020_sept2020_sheet_names = eden_data_functions.get_sheet_names(may2020_sept2020_location)
may2019_sept2019_sheet_names = eden_data_functions.get_sheet_names(may2019_sept2019_location)
other_months_2020_sheet_names = eden_data_functions.get_sheet_names(other_months_2020_location)
other_months_2019_sheet_names = eden_data_functions.get_sheet_names(other_months_2019_location)

# need to drop data from earlier in months where we have double data (e.g., May 10,15 and May 31, keeping just end of month)
may2020_sept2020_sheet_names = [x for x in may2020_sept2020_sheet_names if not "15" in x]
may2020_sept2020_sheet_names = [x for x in may2020_sept2020_sheet_names if not "10" in x]

Load in and merge data 

In [42]:
first_month = eden_data_functions.read_in_check_status(other_months_2019_location,other_months_2019_sheet_names[0]) 

full_data = [first_month, eden_data_functions.read_in(other_months_2019_location,other_months_2019_sheet_names[1]),
           eden_data_functions.read_in(other_months_2019_location,other_months_2019_sheet_names[2]),
           eden_data_functions.read_in(other_months_2019_location,other_months_2019_sheet_names[3]),
           eden_data_functions.read_in(may2019_sept2019_location,may2019_sept2019_sheet_names[0]),
           eden_data_functions.read_in(may2019_sept2019_location,may2019_sept2019_sheet_names[1]),
           eden_data_functions.read_in(may2019_sept2019_location,may2019_sept2019_sheet_names[2]),
           eden_data_functions.read_in(may2019_sept2019_location,may2019_sept2019_sheet_names[3]),
           eden_data_functions.read_in(may2019_sept2019_location,may2019_sept2019_sheet_names[4]),
           eden_data_functions.read_in(other_months_2019_location,other_months_2019_sheet_names[4]),
           eden_data_functions.read_in(other_months_2019_location,other_months_2019_sheet_names[5]),
           eden_data_functions.read_in(other_months_2019_location,other_months_2019_sheet_names[6]),
           #2020
           eden_data_functions.read_in(other_months_2020_location,other_months_2020_sheet_names[1]),
           eden_data_functions.read_in(other_months_2020_location,other_months_2020_sheet_names[1]),
           eden_data_functions.read_in(other_months_2020_location,other_months_2020_sheet_names[2]),
           eden_data_functions.read_in(other_months_2020_location,other_months_2020_sheet_names[3]),
           eden_data_functions.read_in(may2020_sept2020_location,may2020_sept2020_sheet_names[0]),
           eden_data_functions.read_in(may2020_sept2020_location,may2020_sept2020_sheet_names[1]),
           eden_data_functions.read_in(may2020_sept2020_location,may2020_sept2020_sheet_names[2]),
           eden_data_functions.read_in(may2020_sept2020_location,may2020_sept2020_sheet_names[3]),
           eden_data_functions.read_in(may2020_sept2020_location,may2020_sept2020_sheet_names[4]),
           eden_data_functions.read_in(other_months_2020_location,other_months_2020_sheet_names[4]),
           eden_data_functions.read_in(other_months_2020_location,other_months_2020_sheet_names[5]),
           eden_data_functions.read_in(other_months_2020_location,other_months_2020_sheet_names[6])]

In [43]:
#This lambda function then merges all the dataframes from df_list together 
full_eden = reduce(lambda left,right: pd.merge(left,right,on='key'), full_data)

Here I'm calling a preliminary cleaning function. It removes employee units as well as units that were not completed for the entirity of 2019-2020 (found by looking at Eden's website for properties with high vacencies in the data)

In [65]:
def filter_eden_rent_roll(df):
    """
    this function removes employee units as well as units that were not completed for the entirity of 2019-2020 
    (found by looking at Eden's website for properties with high vacencies in the data)
    """
    df = df.drop(columns=["Period"]) #drop col we dont need anymore 
    searchfor = ["employee","manager"]
    df = df[~df.key.str.lower().str.contains('|'.join(searchfor))]
    all_hands_on_the_bad_ones = ['Stone Pine Meadows','Quail Run Apartments','Lincoln Corner Apartments',
    'Vacaville Meadows Drive','Orchard/Maples Apartments','Willows Apartments',
    'Highlands Apartments', 'Hillside Senior Apartments']
    df = df[~df["Property"].isin(all_hands_on_the_bad_ones)]
    return df 

In [67]:
full_eden = eden_data_functions.filter_eden_rent_roll(full_eden)
#full_eden.to_csv("wide_eden_rental_dataset.csv")

In [54]:
#read in Eden residents to see if can match 
eden_residents = pd.read_csv("Eden2018_2019residents.csv")

In [75]:
#clean the data ADD FUNCTION HEADERS 
df_tenants_clean = eden_data_functions.clean_names(full_eden,"Name","Tenant")
eden_residents_clean = eden_data_functions.clean_names(eden_residents,"Member_Name","Tenant_Code",eden_tenants=False)

In [78]:
#filter eden residents by dropping duplicates, data that only exits in 2018, and those that are not heads of households
eden_residents_clean = eden_data_functions.filter_eden_resident(eden_residents_clean)